# Deep Convolutional GAN (DCGAN) with NPY files

# GPU周りの設定

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# -------------------------------------------------------
# メモリの制限 tensorflow-gpu (2.0.0)
# -------------------------------------------------------
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for k in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[k], True)
        print(physical_devices[k])
        print('memory growth:', tf.config.experimental.get_memory_growth(physical_devices[k]))
        print("GPUを使用します")        
else:
    print("GPUが見つかりません")

## 環境変数の設定　状況に合わせて変える必要がある

In [ ]:
# 引き継ぐ学習回数の指定
train_iterates = 0

#扱うデータのサイズ
img_size = 256
channels = 3
# 入力の形状 //カラー画像
img_shape = (img_size, img_size, channels)

# noiseベクトルサイズ（生成器へのINPUT)
z_dim = 100

# 画像の枚数 img_num * img_num枚出てくる
img_num = 3

# 画像のサイズ
fig_size = 16

# GanPracticeディレクトリへのパス
path_to_GanPractice = '/home/nesh202004344/デスクトップ/ぶどうちゃん用フォルダ/GanPractice/'
# デスクトップへのパス
path_to_Desktop = '/home/nesh202004344/デスクトップ/'

## ハイパーパラメータの設定

In [ ]:
#バッヂサイズ
batch_size = 64

#最大反復回数
iterations =  500000

#画像出力の感覚
sample_interval = 500

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
# from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array, array_to_img
#モデルの可視化
#from tensorflow.python.keras.utils.vis_utils import plot_model

from keras.utils.vis_utils import plot_model
from keras.models import load_model	
import glob

## Generator

In [ ]:
def build_generator(z_dim):

    model = Sequential()

    # 全結合層によってnoiseベクトル(200次元）をReshapeして7x7x256 tensorに変換する
    model.add(Dense(256 * 8 * 8, input_dim=z_dim))
    model.add(Reshape((8, 8, 256)))

    # 転置畳み込みにより8x8x256 から 16x16x128テンソルに変換
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))

    # バッチ正規化
    model.add(BatchNormalization())

    # Leaky ReLU活性化
    model.add(LeakyReLU(alpha=0.01))

    # 転置畳み込みにより16x16x128 から 32x32x64テンソルに変換
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))

    # バッチ正規化
    model.add(BatchNormalization())

    # Leaky ReLU活性化
    model.add(LeakyReLU(alpha=0.01))

    # 転置畳み込みにより32x32x64 から32x32x32 テンソルに変換
    model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
    
    # 転置畳み込みにより32x32x32 から64x64x64 テンソルに変換
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))

    # バッチ正規化
    model.add(BatchNormalization())

    # Leaky ReLU活性化
    model.add(LeakyReLU(alpha=0.01))

    # Transposed convolution layer, from 32x32x32 to variable tensor
    stride = int(img_size/64)
    model.add(Conv2DTranspose(3, kernel_size=3, strides=stride, padding='same'))

    # tanh活性化を適用して出力（最終層だけはバッチ正規化はしない）
    model.add(Activation('tanh'))

    return model

In [ ]:
build_model = build_generator(z_dim)

# plot_model(build_model, to_file="DCGAB_build_model.png", show_shapes=True)

## Discriminator

In [ ]:
def build_discriminator(img_shape):

    model = Sequential()

    # 入力画像を32x32x32のテンソルにする畳み込み層
    dis_stride = int(img_size/32)
    model.add(
        Conv2D(32,
               kernel_size=3,
               strides=dis_stride,
               input_shape=img_shape,
               padding='same')) #padding=sameにすると、入力の大きさをstridesの大きさで単純に割ったもの(28/2=14)が出力の大きさになる

    # Leaky ReLUによる活性化(最初の層にはバッチ正規化は適用しない)
    model.add(LeakyReLU(alpha=0.01))

    # 32x32x32 を16x16x64のテンソルにする畳み込み層
    model.add(
        Conv2D(64,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))

    # バッチ正規化
    model.add(BatchNormalization())

    # Leaky ReLUによる活性化
    model.add(LeakyReLU(alpha=0.01))

    # 16x16x64 を8x8x128のテンソルにする畳み込み層
    model.add(
        Conv2D(128,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))

    # バッチ正規化
    model.add(BatchNormalization())

    # Leaky ReLU活性化
    model.add(LeakyReLU(alpha=0.01))

    # シグモイド関数で出力（０～１）
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
discriminator_model = build_discriminator(img_shape)

# plot_model(discriminator_model, to_file="DCGAB_discriminator_model.png", show_shapes=True)

In [ ]:
def build_gan(generator, discriminator):

    model = Sequential()

    # 生成器と識別機を結合
    model.add(generator)
    model.add(discriminator)

    return model

In [ ]:
discriminator = build_discriminator(img_shape)
generator = build_generator(z_dim)
gan_model = build_gan(generator, discriminator)

# plot_model(gan_model, to_file="DCGAB_gan_model.png", show_shapes=True)

In [ ]:
# 識別機の生成とコンパイル
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

#前回学習の重みをロード
if train_iterates > 0:
    weights_path_1 = f'{path_to_GanPractice}dcgan/models/D/d_model-' + str(train_iterates) + '.h5'
    discriminator.load_weights(weights_path_1)

# 生成器の生成
generator = build_generator(z_dim)

# 生成器の訓練時は識別機のパラメータを固定する
discriminator.trainable = False

# 識別機は固定のまま生成器を訓練するGANモデルの生成とコンパイル
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
#途中までの学習済みモデルのロード
from keras.models import load_model

model_g_dir = f"{path_to_GanPractice} + dcgan/models/G"
model_d_dir = f"{path_to_GanPractice} + dcgan/models/D"
if train_iterates > 0:
    models_path_1 = model_g_dir + '/g_param-' + str(train_iterates) + '.hdf5'
    generator.load_weights(models_path_1)

if train_iterates > 0:
    models_path_2 = model_g_dir + '/gan_param-' + str(train_iterates) + '.hdf5'
    gan.load_weights(models_path_2)

## Training

In [ ]:
losses = []
accuracies = []
iteration_checkpoints = []
output_dir = f"{path_to_GanPractice}dcgan/256"
model_g_dir = f"{path_to_GanPractice}dcgan/models/G"
model_d_dir = f"{path_to_GanPractice}dcgan/models/D"
num_of_trials = train_iterates

def train(iterations, batch_size, sample_interval):
  # 学習するデータセットを読み込む
  datasets = np.load(f"{path_to_Desktop}resultnpys/no_rotate/{img_size}/result.npy")
  # ndary型に変換
  X_train = datasets 
  # Rescale [0, 255] grayscale pixel values to [-1, 1]
  X_train = X_train / 127.5 - 1.0
    
  # Labels for real images: all ones
  real = np.ones((batch_size, 1))

  # Labels for fake images: all zeros
  fake = np.zeros((batch_size, 1))
  print("check-001")

  for iteration in range(iterations):

      # -------------------------
      #  Train the Discriminator
      # ------------\img-------------
      print("iteration=", iteration)
      # Get a random batch of real images
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
       # Generate a batch of fake images
      z = np.random.normal(0, 1, (batch_size, 100))
      gen_imgs = generator.predict(z)
       # Train Discriminator
      d_loss_real = discriminator.train_on_batch(imgs, real)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
      d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
     
      
      # ---------------------
      #  Train the Generator
      # ---------------------

      # Generate a batch of fake images
      z = np.random.normal(0, 1, (batch_size, 100))
      gen_imgs = generator.predict(z)

      # Train Generator
      g_loss = gan.train_on_batch(z, real)

      # if (iteration + 1) % sample_interval == 0:
      if iteration % sample_interval == 0:
          # Save losses and accuracies so they can be plotted after training
          losses.append((d_loss, g_loss))
          accuracies.append(100.0 * accuracy)
          iteration_checkpoints.append(iteration + 1)

          # Output training progress
          print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

          # Output a sample of generated image
          sample_images(generator, iteration)
          #識別器のモデル保存
          d_model_file=os.path.join(model_d_dir, "d_model-" + str(iteration + num_of_trials) +  ".h5")
          d_param_file=os.path.join(model_d_dir, "d_param-" + str(iteration + num_of_trials) +  ".hdf5")
          discriminator.save(d_model_file)	
          discriminator.save_weights(d_param_file)
          
          #生成器のモデル保存
          g_model_file=os.path.join(model_g_dir, "g_model-" + str(iteration + num_of_trials) +  ".h5")
          g_param_file=os.path.join(model_g_dir, "g_param-" + str(iteration + num_of_trials) +  ".hdf5")
          generator.save(g_model_file)	
          generator.save_weights(g_param_file)
          gan_model_file=os.path.join(model_g_dir, "gan_model-" + str(iteration + num_of_trials) +  ".h5")
          gan_param_file=os.path.join(model_g_dir, "gan_param-" + str(iteration + num_of_trials) +  ".hdf5")

          gan.save(gan_model_file)	
          gan.save_weights(gan_param_file)
          
          



In [ ]:
def sample_images(generator, iteration, image_grid_rows=img_num, image_grid_columns=img_num):

    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # Generate images from random noise
    gen_imgs = generator.predict(z)  # (16, 64, 64, 3)

    # Rescale image pixel values to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(fig_size, fig_size),
                            sharey=True,
                            sharex=True)
    print("生成されている画像の枚数とサイズ", gen_imgs.shape)
    print("生成されている一枚一枚の画像サイズ=", gen_imgs[0,:,:,:].shape)
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    #file保存
    output_file = os.path.join(output_dir, 'result_' + str(iteration + num_of_trials) +'.png')
    plt.savefig(output_file)


In [ ]:
# Train the DCGAN for the specified number of iterations
train(iterations, batch_size, sample_interval)